<a href="https://colab.research.google.com/github/onoregleb/SFN/blob/main/%D0%9C%D0%B0%D0%B3_%D0%B4%D0%B8%D1%81%D1%81_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download odins0n/ucf-crime-dataset

Dataset URL: https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset
License(s): CC0-1.0
100% 11.0G/11.0G [06:47<00:00, 30.8MB/s]
100% 11.0G/11.0G [06:47<00:00, 29.0MB/s]


In [7]:
! unzip ucf-crime-dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: Train/Vandalism/Vandalism035_x264_230.png  
  inflating: Train/Vandalism/Vandalism035_x264_240.png  
  inflating: Train/Vandalism/Vandalism035_x264_250.png  
  inflating: Train/Vandalism/Vandalism035_x264_260.png  
  inflating: Train/Vandalism/Vandalism035_x264_270.png  
  inflating: Train/Vandalism/Vandalism035_x264_280.png  
  inflating: Train/Vandalism/Vandalism035_x264_290.png  
  inflating: Train/Vandalism/Vandalism035_x264_30.png  
  inflating: Train/Vandalism/Vandalism035_x264_300.png  
  inflating: Train/Vandalism/Vandalism035_x264_310.png  
  inflating: Train/Vandalism/Vandalism035_x264_320.png  
  inflating: Train/Vandalism/Vandalism035_x264_330.png  
  inflating: Train/Vandalism/Vandalism035_x264_340.png  
  inflating: Train/Vandalism/Vandalism035_x264_350.png  
  inflating: Train/Vandalism/Vandalism035_x264_360.png  
  inflating: Train/Vandalism/Vandalism035_x264_370.png  
  inflating: Train/Va

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score


from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_dir = "Train"
test_dir = "Test"

In [ ]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_fun
                                 )

In [ ]:
SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 512
EPOCHS = 3
LR = 0.0003
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion',
                'Fighting',"Normal",'RoadAccidents','Robbery','Shooting',
                'Shoplifting','Stealing','Vandalism']

In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    color_mode="rgb",
                                                    class_mode="categorical",
                                                    seed=SEED
                                                   )
test_generator = test_datagen.flow_from_directory(directory=test_dir,
                                                  target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                  batch_size=BATCH_SIZE,
                                                  shuffle=False,
                                                  color_mode="rgb",
                                                  class_mode="categorical",
                                                  seed=SEED
                                                 )

Found 1266345 images belonging to 14 classes.
Found 111308 images belonging to 14 classes.


In [ ]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.VGG16(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                                     include_top=False,
                                                     weights="imagenet")(inputs)
    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    return x

def final_model(inputs):
    vgg16_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(vgg16_feature_extractor)
    return classification_output

def define_compile_model():
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    classification_output = final_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=classification_output)
    model.compile(optimizer=tf.keras.optimizers.SGD(LR),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = define_compile_model()
clear_output()
model.summary()

48914432/58889256 [=======================>......] - ETA: 0s

KeyboardInterrupt: 

In [ ]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

class CustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.accuracy = []

    def on_epoch_end(self, epoch, logs=None):
        print("Epoch:", epoch + 1)
        print("Loss:", logs['loss'])
        print("accuracy:", logs['accuracy'])

    def on_train_batch_end(self, batch, logs=None):
        print("\nBatch:", batch + 1)
        print("Loss:", logs['loss'])
        print("accuracy:", logs['accuracy'])
        self.losses.append(logs['loss'])
        self.accuracy.append(logs['accuracy'])
        self.plot_metrics()

    def plot_metrics(self):
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(self.losses)
        plt.title('Loss')
        plt.subplot(1, 2, 2)
        plt.plot(self.accuracy)
        plt.title('accuracy')
        plt.show()

custom_callback = CustomCallback()

In [ ]:
history = model.fit(x=train_generator,
                    validation_data=test_generator,
                    epochs=EPOCHS,
                    callbacks=[custom_callback])

##SFN

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

train_dir = "Train"
test_dir = "Test"

preprocess_fun = tf.keras.applications.densenet.preprocess_input

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 512
EPOCHS = 3
LR = 0.0003
NUM_CLASSES = 14
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion',
                'Fighting',"Normal",'RoadAccidents','Robbery','Shooting',
                'Shoplifting','Stealing','Vandalism']

# Создание двух генераторов данных: одного для медленного пути, другого для быстрого пути
train_datagen_slow = ImageDataGenerator(horizontal_flip=True,
                                        width_shift_range=0.1,
                                        height_shift_range=0.05,
                                        rescale = 1./255,
                                        preprocessing_function=preprocess_fun
                                       )
train_datagen_fast = ImageDataGenerator(horizontal_flip=True,
                                        width_shift_range=0.1,
                                        height_shift_range=0.05,
                                        rescale = 1./255,
                                        preprocessing_function=preprocess_fun
                                       )

test_datagen_slow = ImageDataGenerator(rescale = 1./255,
                                       preprocessing_function=preprocess_fun
                                      )
test_datagen_fast = ImageDataGenerator(rescale = 1./255,
                                       preprocessing_function=preprocess_fun
                                      )

train_generator_slow = train_datagen_slow.flow_from_directory(directory=train_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              batch_size=BATCH_SIZE,
                                                              shuffle=True,
                                                              color_mode="rgb",
                                                              class_mode="categorical",
                                                              seed=SEED
                                                             )
train_generator_fast = train_datagen_fast.flow_from_directory(directory=train_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              batch_size=BATCH_SIZE,
                                                              shuffle=True,
                                                              color_mode="rgb",
                                                              class_mode="categorical",
                                                              seed=SEED
                                                             )

test_generator_slow = test_datagen_slow.flow_from_directory(directory=test_dir,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            batch_size=BATCH_SIZE,
                                                            shuffle=False,
                                                            color_mode="rgb",
                                                            class_mode="categorical",
                                                            seed=SEED
                                                           )
test_generator_fast = test_datagen_fast.flow_from_directory(directory=test_dir,
                                                            target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                            batch_size=BATCH_SIZE,
                                                            shuffle=False,
                                                            color_mode="rgb",
                                                            class_mode="categorical",
                                                            seed=SEED
                                                           )


Found 1266345 images belonging to 14 classes.
Found 1266345 images belonging to 14 classes.
Found 111308 images belonging to 14 classes.
Found 111308 images belonging to 14 classes.


In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model

# Определение архитектуры SlowFast Network
def slowfast_model(input_shape, num_classes):
    # Входные данные для медленного пути
    slow_input = Input(shape=input_shape)
    # Сверточные слои медленного пути
    slow = Conv2D(64, (3, 3), activation='relu', padding='same')(slow_input)
    slow = MaxPooling2D(pool_size=(2, 2))(slow)
    # Добавьте здесь больше слоев для медленного пути

    # Входные данные для быстрого пути
    fast_input = Input(shape=input_shape)
    # Сверточные слои быстрого пути
    fast = Conv2D(64, (3, 3), activation='relu', padding='same')(fast_input)
    fast = MaxPooling2D(pool_size=(2, 2))(fast)
    # Добавьте здесь больше слоев для быстрого пути

    # Объединение выходов медленного и быстрого путей
    combined = concatenate([slow, fast])

    # Полносвязные слои
    x = Flatten()(combined)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # Создание модели
    model = Model(inputs=[slow_input, fast_input], outputs=outputs)
    return model

# Создание экземпляра модели
model = slowfast_model(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=NUM_CLASSES)

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [11]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

class CustomCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.accuracy = []

    def on_epoch_end(self, epoch, logs=None):
        print("Epoch:", epoch + 1)
        print("Loss:", logs['loss'])
        print("accuracy:", logs['accuracy'])

    def on_train_batch_end(self, batch, logs=None):
        print("\nBatch:", batch + 1)
        print("Loss:", logs['loss'])
        print("accuracy:", logs['accuracy'])
        self.losses.append(logs['loss'])
        self.accuracy.append(logs['accuracy'])
        self.plot_metrics()

    def plot_metrics(self):
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(self.losses)
        plt.title('Loss')
        plt.subplot(1, 2, 2)
        plt.plot(self.accuracy)
        plt.title('accuracy')
        plt.show()

custom_callback = CustomCallback()

In [ ]:
def combine_generators(gen1, gen2):
    while True:
        yield [next(gen1)[0], next(gen2)[0]], next(gen1)[1]

# Создание объединенных генераторов для обучения и тестирования
train_generator_combined = combine_generators(train_generator_slow, train_generator_fast)
test_generator_combined = combine_generators(test_generator_slow, test_generator_fast)

# Обучение модели
history = model.fit(x=train_generator_combined,
                    validation_data=test_generator_combined,
                    steps_per_epoch=len(train_generator_slow),
                    validation_steps=len(test_generator_slow),
                    epochs=EPOCHS, callbacks = custom_callback)


In [ ]:
from sklearn.metrics import roc_auc_score, classification_report
import matplotlib.pyplot as plt

# Предсказание на тестовом наборе
y_pred = model.predict(test_generator_combined, steps=len(test_generator_slow))

# Получение истинных меток из генератора
y_true = test_generator_slow.classes

# Расчет ROC AUC
roc_auc = roc_auc_score(y_true, y_pred)
print('ROC AUC: {0}'.format(roc_auc))

# Вывод отчета о классификации
print(classification_report(y_true, np.argmax(y_pred, axis=1)))

# Построение графика ROC для каждого класса
plt.figure(figsize=(10, 10))
for i in range(NUM_CLASSES):
    fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
    plt.plot(fpr, tpr, label='class %d (AUC=%.4f)' % (i, auc(fpr, tpr)))
plt.legend()
plt.title('ROC curves')
plt.show()
